In [1]:
from eptnr.rewards.egalitarian import EgalitarianTheilReward, BaseReward
from eptnr.constants.travel_metric import TravelMetric

from eptnr.analysis.full_problem_exploration import full_problem_exploration
from eptnr.plotting.solution_plotting import plot_full_problem_exploration
from inequality.theil import TheilD
import igraph as ig
import numpy as np

In [2]:
class InverseTheilReward(BaseReward):
    def _evaluate(self, g: ig.Graph, *args, **kwargs) -> float:
        """

        Args:
            g:

        Returns:

        """
        metrics_dfs = self.retrieve_dfs(g)

        theil_inequality = {metric: None for metric in self.metrics_names}

        for metric, metric_df in metrics_dfs.items():
            X = metric_df.drop(columns='group').astype(float).to_numpy()
            Y = metric_df.group.to_numpy()
            theil_t = TheilD(X, Y).T[0] if X.sum() > 0 else 0.0
            theil_inequality[metric] = theil_t

        return sum(theil_inequality.values())

    def _reward_scaling(self, reward: float) -> float:
        return np.log(len(self.census_data)) - reward